Importação das bibliotecas

In [15]:
import pandas as pd
# import numpy as np
import statsmodels.api as sm
# import matplotlib.pyplot as plt
# import seaborn as sns
# import pingouin as pg
# from statstests.process import stepwise

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.10 from "c:\Users\vinicius.oliveira\AppData\Local\Programs\Python\Python310\python.exe"
  * The NumPy version is: "1.23.1"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: Não foi possível encontrar o módulo especificado.


ImportError: numpy._core.multiarray failed to import

Exploração dos dados

In [ ]:
# Lendo arquivo
dataset = pd.read_excel('./tempo_distancia.xlsx')
dataset.head()

,estudante,tempo,distancia,semaforos,periodo,perfil
0,Gabriela,15,8,0,manhã,calmo
1,Dalila,20,6,1,manhã,moderado
2,Gustavo,20,15,0,manhã,moderado
3,Letícia,40,20,1,tarde,agressivo
4,Luiz Ovídio,50,25,2,tarde,agressivo


In [ ]:
# Métricas das variáveis quantitativas
dataset.drop(columns=['estudante', 'periodo', 'perfil']).describe()

,tempo,distancia,semaforos
count,10.00000,10.000000,10.000000
mean,30.00000,17.000000,1.000000
std,14.90712,9.510228,0.942809
min,10.00000,5.000000,0.000000
25%,20.00000,8.750000,0.250000
50%,27.50000,17.500000,1.000000
75%,38.75000,23.750000,1.000000
max,55.00000,32.000000,3.000000


In [ ]:
# Visualizando matriz de correlação
dataset.corr()

,tempo,distancia,semaforos
tempo,1.000000,0.905221,0.909155
distancia,0.905221,1.000000,0.755914
semaforos,0.909155,0.755914,1.000000


In [ ]:
# Contagem por agrupamento de valores de variáveis qualitativas
print('Período:')
print(dataset['periodo'].value_counts())
print('\nPerfil:')
print(dataset['perfil'].value_counts())

Período:
manhã    7
tarde    3
Name: periodo, dtype: int64

Perfil:
moderado     5
calmo        3
agressivo    2
Name: perfil, dtype: int64


In [ ]:
# Removendo linhas que contenham algum valor nulo
dataset.dropna(inplace=True)

In [ ]:
# Realizando o processo de dummy nas variáveis qualitativas
dataset = pd.get_dummies(data=dataset
                         , columns=['periodo', 'perfil']
                         , drop_first=True
                         , dtype='float')
dataset.head()

,estudante,tempo,distancia,semaforos,periodo_tarde,perfil_calmo,perfil_moderado
0,Gabriela,15,8,0,0.0,1.0,0.0
1,Dalila,20,6,1,0.0,0.0,1.0
2,Gustavo,20,15,0,0.0,0.0,1.0
3,Letícia,40,20,1,1.0,0.0,0.0
4,Luiz Ovídio,50,25,2,1.0,0.0,0.0


Geração do modelo

In [ ]:
# Função para geração do texto da fórmula
def texto_formula(df, var_dependente, excluir_cols):
    """
        Função para gerar o texto utilizado na 
        fórmula para geração do modelo
    """
    variaveis = list(df.columns.values)
    variaveis.remove(var_dependente)
    for col in excluir_cols:
        variaveis.remove(col)
    return var_dependente + ' ~ ' + ' + '.join(variaveis)

In [ ]:
# Gerando o modelo
reg = 